<a href="https://colab.research.google.com/github/sushirito/Molecular-Dynamics/blob/main/Copy_of_OpenMM_Scripting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**NOTE**: Code must be run twice to work

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

!mamba install -c conda-forge ambertools -y

✨🍰✨ Everything looks OK!

Looking for: ['ambertools']

conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache

Pinned packages:
  - python 3.11.*
  - python 3.11.*
  - python_abi 3.11.* *cp311*
  - cuda-version 12.*


Transaction

  Prefix: /usr/local

  All requested packages already installed



In [5]:
!git clone https://github.com/Iourarum/GOPY.git
%cd GOPY
!python GOPY.py generate_PG 10 10 graphene.pdb
!python GOPY.py generate_GO graphene.pdb 20 0 0 functionalized_carboxyl.pdb
!python GOPY.py generate_GO graphene.pdb 0 0 20 functionalized_hydroxyl.pdb
!python GOPY.py generate_GO graphene.pdb 0 20 0 functionalized_epoxy.pdb

Cloning into 'GOPY'...
remote: Enumerating objects: 197, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 197 (delta 39), reused 18 (delta 18), pack-reused 150 (from 1)
Receiving objects: 100% (197/197), 9.50 MiB | 25.47 MiB/s, done.
Resolving deltas: 100% (99/99), done.
/content/GOPY/GOPY
done.
GOPY.py
generate_GO
graphene.pdb
20
0
0
functionalized_carboxyl.pdb
Left to add:  cooh:  20 epoxy:  0 hydroxyl:  0
Left to add:  cooh:  19 epoxy:  0 hydroxyl:  0
Left to add:  cooh:  18 epoxy:  0 hydroxyl:  0
Left to add:  cooh:  17 epoxy:  0 hydroxyl:  0
Left to add:  cooh:  16 epoxy:  0 hydroxyl:  0
Left to add:  cooh:  15 epoxy:  0 hydroxyl:  0
Left to add:  cooh:  14 epoxy:  0 hydroxyl:  0
Left to add:  cooh:  13 epoxy:  0 hydroxyl:  0
Left to add:  cooh:  12 epoxy:  0 hydroxyl:  0
Left to add:  cooh:  11 epoxy:  0 hydroxyl:  0
Left to add:  cooh:  10 epoxy:  0 hydroxyl:  0
Left to add:  cooh:  9 epoxy:  0 hydroxyl:  0
Left t

In [3]:
!wget https://raw.githubusercontent.com/Iourarum/GOPY/master/GO_tutorial/GGG.lib
!wget https://raw.githubusercontent.com/Iourarum/GOPY/master/GO_tutorial/C1A.lib
!wget https://raw.githubusercontent.com/Iourarum/GOPY/master/GO_tutorial/E1A.lib
!wget https://raw.githubusercontent.com/Iourarum/GOPY/master/GO_tutorial/H1A.lib
!mkdir -p FRCMOD
!wget -P FRCMOD https://raw.githubusercontent.com/Iourarum/GOPY/master/GO_tutorial/FRCMOD/GO.frcmod

--2025-03-07 23:00:07--  https://raw.githubusercontent.com/Iourarum/GOPY/master/GO_tutorial/GGG.lib
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1063 (1.0K) [text/plain]
Saving to: ‘GGG.lib’

GGG.lib             100%[===================>]   1.04K  --.-KB/s    in 0s      

2025-03-07 23:00:07 (44.3 MB/s) - ‘GGG.lib’ saved [1063/1063]

--2025-03-07 23:00:08--  https://raw.githubusercontent.com/Iourarum/GOPY/master/GO_tutorial/C1A.lib
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1627 (1.6K) [text/plain]
Saving to: ‘C1A.lib’

C1

In [4]:
!pip install openmm
!pip install MDAnalysis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 74.5 MB/s eta 0:00:00


In [6]:
import os, numpy as np, matplotlib.pyplot as plt, csv
import MDAnalysis as mda
from MDAnalysis.analysis.rdf import InterRDF
from scipy.signal import find_peaks
from scipy.spatial import cKDTree
from scipy.optimize import curve_fit
from openmm.app import AmberPrmtopFile, AmberInpcrdFile, Simulation, PME, HBonds, DCDReporter
from openmm import LangevinMiddleIntegrator, Platform
from openmm.unit import kelvin, picosecond, nanometer

# ----- Global Simulation Parameters -----
BOX_DIMS = (150.0, 150.0, 100.0)  # in Å
SOLVATION_BOX = 5.0               # padding (Å)
NUM_STEPS = 5000                  # simulation steps
REPORT_INTERVAL = 100             # trajectory saving interval

# List of AC types to compare; assume corresponding PDB files exist:
ac_types = {
    'epoxy': 'functionalized_epoxy.pdb',
    'hydroxyl': 'functionalized_hydroxyl.pdb',
    'carboxyl': 'functionalized_carboxyl.pdb'
}

# Mercury ion count (set a fixed number for simplicity)
HG_COUNT = 50

# ----- Function to Prepare & Run Simulation -----
def run_simulation(ac_type, pdb_file, hg_count=HG_COUNT):
    # Write tleap input (adjust ion counts as needed)
    tleap_in = f"""\
source leaprc.protein.ff14SB
source leaprc.water.tip3p
loadAmberParams frcmod.ions234lm_126_tip3p
loadoff GGG.lib
loadoff C1A.lib
loadoff E1A.lib
loadoff H1A.lib
loadamberparams FRCMOD/GO.frcmod
mol = loadpdb {pdb_file}
bondbydistance mol
solvateBox mol TIP3PBOX {SOLVATION_BOX}
addIons2 mol HG {hg_count}
addIons2 mol Cl- {2*hg_count}
saveAmberParm mol mol_solv.prmtop mol_solv.inpcrd
quit
"""
    with open('tleap.in', 'w') as f:
        f.write(tleap_in)
    os.system('tleap -f tleap.in')

    # Setup simulation with OpenMM
    prmtop = AmberPrmtopFile('mol_solv.prmtop')
    inpcrd = AmberInpcrdFile('mol_solv.inpcrd')
    system = prmtop.createSystem(nonbondedMethod=PME,
                                 nonbondedCutoff=1.0*nanometer,
                                 constraints=HBonds)
    integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picosecond)
    simulation = Simulation(prmtop.topology, system, integrator, Platform.getPlatformByName('CPU'))
    simulation.context.setPositions(inpcrd.positions)
    simulation.minimizeEnergy()
    simulation.reporters.append(DCDReporter('trajectory.dcd', REPORT_INTERVAL))
    simulation.step(NUM_STEPS)
    return 'mol_solv.prmtop', 'trajectory.dcd'

# ----- Analysis Functions -----
def analyze_rdf(u, group_AC, group_HG):
    rdf_calc = InterRDF(group_AC, group_HG, range=(0,12.0), nbins=120)
    rdf_calc.run()
    # Find first minimum after first peak for cutoff
    peaks, _ = find_peaks(rdf_calc.rdf)
    if len(peaks) > 0:
        first_peak = peaks[0]
        inverted = -rdf_calc.rdf
        minima, _ = find_peaks(inverted)
        minima_after = [m for m in minima if m > first_peak]
        r_cut = rdf_calc.bins[minima_after[0]] if minima_after else 12.0
    else:
        r_cut = 12.0
    # Coordination number via integration
    dr = rdf_calc.bins[1]-rdf_calc.bins[0]
    # Assume bulk density = number of HG atoms / box volume (in Å³)
    density = len(group_HG) / np.prod(u.dimensions[:3])
    coord_num = np.sum(4*np.pi*(rdf_calc.bins**2)*rdf_calc.rdf*dr)*density
    return rdf_calc.bins, rdf_calc.rdf, r_cut, coord_num

def analyze_density_profile(u, group_HG, axis='z', bins=50):
    # Compute atomic density profile along chosen axis
    coords = group_HG.positions[:, 2]  # z-axis
    hist, edges = np.histogram(coords, bins=bins, density=True)
    centers = (edges[:-1] + edges[1:]) / 2
    return centers, hist

def analyze_msd(u, group_HG):
    # Compute MSD of mercury ions over trajectory
    positions0 = group_HG.positions.copy()
    msd_list = []
    times = []
    dt = 0.004  # ps per step (from integrator)
    for ts in u.trajectory:
        disp = group_HG.positions - positions0
        msd = np.mean(np.sum(disp**2, axis=1))
        msd_list.append(msd)
        times.append(ts.time)  # time in ps
    times = np.array(times)
    msd_list = np.array(msd_list)
    # Linear fit (fit over second half for better statistics)
    fit_mask = times > (times[-1]*0.5)
    slope, _ = np.polyfit(times[fit_mask], msd_list[fit_mask], 1)
    # Diffusion coefficient: D = slope / (2*d) with d=3
    D = slope / (2*3)
    return times, msd_list, D

# ----- Main Execution Loop -----
results = {}
for ac_label, pdb_file in ac_types.items():
    print(f"Processing AC type: {ac_label}")
    # Run simulation
    top_file, traj_file = run_simulation(ac_label, pdb_file)

    # Load trajectory
    u = mda.Universe(top_file, traj_file)
    # Set box dimensions for each frame
    for ts in u.trajectory:
        ts.dimensions = [*BOX_DIMS, 90, 90, 90]
    u.trajectory.rewind()

    # Define groups: assume adsorbent residues are labeled 'AC' (or use a custom selection)
    # Adjust selection strings as needed
    group_AC = u.select_atoms("not name HG and not resname HOH and not resname Cl")
    group_HG = u.select_atoms("name HG")

    # Compute RDF & coordination number
    r_bins, rdf_vals, r_cut, coord_num = analyze_rdf(u, group_AC, group_HG)

    # Compute atomic density profile (for mercury along z)
    dens_z, density_vals = analyze_density_profile(u, group_HG)

    # Compute MSD & diffusion coefficient
    times, msd_vals, D = analyze_msd(u, group_HG)

    # Save results for later comparison
    results[ac_label] = {
        'r_bins': r_bins, 'rdf': rdf_vals, 'r_cut': r_cut, 'coordination': coord_num,
        'density_profile': (dens_z, density_vals),
        'msd': (times, msd_vals), 'diffusion': D
    }

    # --- Plotting for this AC type ---
    plt.figure(figsize=(6,4))
    plt.plot(r_bins, rdf_vals)
    plt.xlabel("r (Å)")
    plt.ylabel("g(r)")
    plt.title(f"RDF for {ac_label}")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(6,4))
    plt.bar(dens_z, density_vals, width=(dens_z[1]-dens_z[0]))
    plt.xlabel("z (Å)")
    plt.ylabel("Density")
    plt.title(f"Atomic Density Profile (HG) for {ac_label}")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(6,4))
    plt.plot(times, msd_vals, 'o-', label=f"MSD, D={D:.3e} Å²/ps")
    plt.xlabel("Time (ps)")
    plt.ylabel("MSD (Å²)")
    plt.title(f"MSD of HG for {ac_label}")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# ----- Comparison Plot -----
plt.figure(figsize=(8,6))
for ac_label in results:
    plt.plot(results[ac_label]['r_bins'], results[ac_label]['rdf'], label=ac_label)
plt.xlabel("r (Å)")
plt.ylabel("g(r)")
plt.title("Comparison of RDFs for Different AC Types")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Summarize metrics
print("Summary of Results:")
for ac_label, res in results.items():
    print(f"AC type: {ac_label}")
    print(f"  Coordination Number: {res['coordination']:.2f}")
    print(f"  Diffusion Coefficient: {res['diffusion']:.3e} Å²/ps")

Processing AC type: epoxy


FileNotFoundError: [Errno 2] No such file or directory: 'mol_solv.prmtop'